#### Project CCF pySpark RDD

In [2]:
r=sc.parallelize([("A","B"),("B","C"),("B","D"),("D","E"),("F","G"),("G","H")],2)
r.collect()

Out[41]: [('A', 'B'), ('B', 'C'), ('B', 'D'), ('D', 'E'), ('F', 'G'), ('G', 'H')]

In [3]:
from itertools import islice
g=sc.textFile("FileStore/tables/webGoogleS.txt")
#remove header 4 lines
data= g.mapPartitionsWithIndex(lambda idx, it: islice(it, 4, None) if idx == 0 else it)
data.count()

Out[24]: 41306

In [4]:
p = data.map(lambda x: x.split("\t")).map(lambda x: (x[0], x[1]))
p.take(5)

Out[25]: [('0', '11342'),
 ('0', '824020'),
 ('0', '867923'),
 ('0', '891835'),
 ('11342', '0')]

In [5]:
def CCF_Iterate_reduce(pair):
  key, values = pair
#  global accum
  min = key
  valueL = []
  for value in values:
    if value < min:
       min = value
    valueL.append(value)
  if min < key:
    yield((key, min))
    for value in valueL:
      if min != value:
        accum.add(1)
        yield((value, min))

In [6]:
log4jLogger = sc._jvm.org.apache.log4j
LOGGER = log4jLogger.LogManager.getLogger(__name__)

In [7]:

new_pair_flag = True
iteration = 0
accum = sc.accumulator(0)
dedupJob = p

In [8]:
#for secondary sort

from pyspark.rdd import portable_hash
import itertools

n = 2 # number to hash
def partitioner(n):
   # Partition by the first item in the key tuple
    def partitioner_(x):
        return portable_hash(x[0]) % n
    return partitioner_


def unpair2(entry):
    return entry[0][0], entry[0][1]
  
def sorted_group(lines):
    return itertools.groupby(lines, key=lambda x: x[0])
  
def CCF_Iterate_reduce_SS_shuffle_test(pair):
  key, values = pair
  minvaluepair = next(values)
  minvalue=minvaluepair[1]
  
  if minvalue < key:
    #return key, minvalue
    yield key, minvalue
    for value in values:
      if minvalue != value[1]:
        accum.add(1)
        yield value[1], minvalue


In [9]:
#### sans secondary sort

new_pair_flag = True
iteration = 0
accum = sc.accumulator(0)
dedupJob = p

while new_pair_flag:
    iteration += 1
    newPair = False
    accum.value = 0

    # CCF-iterate (MAP)
    mapJob = dedupJob.flatMap(lambda e: (e,e[::-1]))
    #print(mapJob.collect())
    
    # CCF-iterate (REDUCE)
    reduceJob = mapJob.groupByKey().flatMap(lambda pair: CCF_Iterate_reduce(pair)).sortByKey()
    
    # CCF-dedup 
    dedupJob = reduceJob.distinct()

    # Force the RDD evalusation
    tmp = dedupJob.count()
    
    # Prepare next iteration
    new_pair_flag = bool(accum.value)

    #LOGGER.warn("Iteration :"+str(iteration)+" --- "+" newPair : "+str(new_pair_flag))
    print("Iteration :"+str(iteration)+" --- "+" newPair : "+str(new_pair_flag) + str(accum.value))
    #for analyse
    results = list(dedupJob.map(lambda e: e[::-1]).groupByKey().map(lambda x : (x[0],tuple(x[1]))).collect())
for k in results:
    print("Component id: "+str(k[0])+"| Number of nodes= "+str(len(k[1])+1))

Iteration :1 --- newPair : True178560
Iteration :2 --- newPair : True94737
Iteration :3 --- newPair : True87195
Iteration :4 --- newPair : True105099
Iteration :5 --- newPair : True89907
Iteration :6 --- newPair : True27543
Iteration :7 --- newPair : True936
Iteration :8 --- newPair : False0
Component id: 0| Number of nodes= 12514
Component id: 101955| Number of nodes= 76
Component id: 102612| Number of nodes= 226
Component id: 101821| Number of nodes= 10
Component id: 107267| Number of nodes= 94
Component id: 105322| Number of nodes= 38
Component id: 108339| Number of nodes= 76
Component id: 111687| Number of nodes= 109
Component id: 113066| Number of nodes= 5
Component id: 108455| Number of nodes= 92
Component id: 122| Number of nodes= 16
Component id: 136602| Number of nodes= 8
Component id: 102704| Number of nodes= 110
Component id: 13978| Number of nodes= 41
Component id: 119117| Number of nodes= 15
Component id: 129131| Number of nodes= 22
Component id: 134769| Number of nodes= 41
Component id: 12420| Number of nodes= 15
Component id: 107| Number of nodes= 19
Component id: 101209| Number of nodes= 11
Component id: 106688| Number of nodes= 16
Component id: 103636| Number of nodes= 21
Component id: 118851| Number of nodes= 7
Component id: 119755| Number of nodes= 6
Component id: 154| Number of nodes= 2
Component id: 120805| Number of nodes= 12
Component id: 171761| Number of nodes= 17
Component id: 161143| Number of nodes= 27
Component id: 108692| Number of nodes= 35
Component id: 129498| Number of nodes= 20
Component id: 111011| Number of nodes= 23
Component id: 165007| Number of nodes= 6
Component id: 113487| Number of nodes= 20
Component id: 128993| Number of nodes= 25
Component id: 112702| Number of nodes= 23
Component id: 129991| Number of nodes= 12
Component id: 124049| Number of nodes= 11
Component id: 130870| Number of nodes= 29
Component id: 213| Number of nodes= 12
Component id: 105031| Number of nodes= 13
Component id: 192473| Number of nodes= 11
Component id: 118607| Number of nodes= 37
Component id: 20779| Number of nodes= 12
Component id: 223980| Number of nodes= 8
Component id: 21695| Number of nodes= 20
Component id: 140813| Number of nodes= 20
Component id: 144930| Number of nodes= 19
Component id: 187829| Number of nodes= 3
Component id: 161520| Number of nodes= 14
Component id: 218961| Number of nodes= 3
Component id: 119| Number of nodes= 9
Component id: 206333| Number of nodes= 6
Component id: 173970| Number of nodes= 11
Component id: 164| Number of nodes= 5
Component id: 156662| Number of nodes= 8
Component id: 11314| Number of nodes= 28
Component id: 243135| Number of nodes= 8
Component id: 219931| Number of nodes= 10
Component id: 186992| Number of nodes= 6
Component id: 399424| Number of nodes= 3
Component id: 102315| Number of nodes= 16
Component id: 102| Number of nodes= 5
Component id: 149930| Number of nodes= 20
Component id: 461| Number of nodes= 3
Component id: 127959| Number of nodes= 3
Component id: 513| Number of nodes= 5
Component id: 338| Number of nodes= 4
Component id: 279176| Number of nodes= 4
Component id: 502| Number of nodes= 3
Component id: 138| Number of nodes= 6
Component id: 321918| Number of nodes= 5
Component id: 145654| Number of nodes= 6
Component id: 167| Number of nodes= 2
Component id: 570| Number of nodes= 3
Component id: 411| Number of nodes= 4
Component id: 272| Number of nodes= 2
Component id: 12| Number of nodes= 2
Component id: 453| Number of nodes= 2
Component id: 270| Number of nodes= 3
Component id: 158318| Number of nodes= 3
Component id: 207025| Number of nodes= 2
Component id: 261310| Number of nodes= 2
Component id: 160| Number of nodes= 2
Component id: 189| Number of nodes= 2
Component id: 365| Number of nodes= 2
Component id: 582910| Number of nodes= 4
Component id: 509| Number of nodes= 2
Component id: 361| Number of nodes= 2
Component id: 100049| Number of nodes= 244
Component id: 102817| Number of nodes= 248
Component id: 101793| Number of nodes= 170
C

In [10]:
#### avec secondary sort

new_pair_flag = True
iteration = 0
accum = sc.accumulator(0)
dedupJob = p

def key_func(entry):
    return entry[0], entry[1]

while new_pair_flag:
    iteration += 1
    newPair = False
    accum.value = 0

    # CCF-iterate (MAP)
    mapJob = dedupJob.flatMap(lambda e: (e,e[::-1]))
    #print(mapJob.collect())
    
    #Secondary Sort
    rddSS= (mapJob
      .keyBy(lambda kv: (kv[0], kv[1]))  # Create temporary composite key
      .repartitionAndSortWithinPartitions(numPartitions=n, partitionFunc=partitioner(n), keyfunc=key_func, ascending=True))
  #.map(lambda x: x[1]))  # Drop key (note: there is no partitioner set anymore)
    #for Secondary sort
    unpairedRDD = rddSS.map(unpair2, preservesPartitioning=True)
    groupedRDD = unpairedRDD.mapPartitions(sorted_group, preservesPartitioning=True)
    reduceJob = groupedRDD.flatMap(CCF_Iterate_reduce_SS_shuffle_test)
    # CCF-iterate (REDUCE)
   # reduceJob = rddSS.groupByKey().flatMap(lambda pair: CCF_Iterate_reduce_SS_shuffle(pair))#.sortByKey()
   # reduceJob = rddSS.groupByKey().flatMap(lambda pair: CCF_Iterate_reduce(pair))#.sortByKey()
    
  # CCF-dedup 
    dedupJob = reduceJob.distinct()

    # Force the RDD evalusation
    tmp = dedupJob.count()
    #print(dedupJob.collect())
    # Prepare next iteration
    #mapJob = dedupJob
    new_pair_flag = bool(accum.value)

    #LOGGER.warn("Iteration :"+str(iteration)+" --- "+" newPair : "+str(new_pair_flag))
    print("Iteration :"+str(iteration)+" --- "+" newPair : "+str(new_pair_flag) + str(accum.value))
    #for analyse
    results = list(dedupJob.map(lambda e: e[::-1]).groupByKey().map(lambda x : (x[0],tuple(x[1]))).collect())
for k in results:
    print("Component id: "+str(k[0])+"| Number of nodes= "+str(len(k[1])+1))

Iteration :1 --- newPair : True59520
Iteration :2 --- newPair : True31579
Iteration :3 --- newPair : True29065
Iteration :4 --- newPair : True35033
Iteration :5 --- newPair : True29969
Iteration :6 --- newPair : True9181
Iteration :7 --- newPair : True312
Iteration :8 --- newPair : False0
Component id: 0| Number of nodes= 12514
Component id: 101955| Number of nodes= 76
Component id: 102612| Number of nodes= 226
Component id: 101821| Number of nodes= 10
Component id: 107267| Number of nodes= 94
Component id: 105322| Number of nodes= 38
Component id: 108339| Number of nodes= 76
Component id: 111687| Number of nodes= 109
Component id: 113066| Number of nodes= 5
Component id: 108455| Number of nodes= 92
Component id: 122| Number of nodes= 16
Component id: 136602| Number of nodes= 8
Component id: 102704| Number of nodes= 110
Component id: 13978| Number of nodes= 41
Component id: 119117| Number of nodes= 15
Component id: 129131| Number of nodes= 22
Component id: 134769| Number of nodes= 41
Component id: 12420| Number of nodes= 15
Component id: 107| Number of nodes= 19
Component id: 101209| Number of nodes= 11
Component id: 106688| Number of nodes= 16
Component id: 103636| Number of nodes= 21
Component id: 118851| Number of nodes= 7
Component id: 119755| Number of nodes= 6
Component id: 154| Number of nodes= 2
Component id: 120805| Number of nodes= 12
Component id: 171761| Number of nodes= 17
Component id: 161143| Number of nodes= 27
Component id: 108692| Number of nodes= 35
Component id: 129498| Number of nodes= 20
Component id: 111011| Number of nodes= 23
Component id: 165007| Number of nodes= 6
Component id: 113487| Number of nodes= 20
Component id: 128993| Number of nodes= 25
Component id: 112702| Number of nodes= 23
Component id: 129991| Number of nodes= 12
Component id: 124049| Number of nodes= 11
Component id: 130870| Number of nodes= 29
Component id: 213| Number of nodes= 12
Component id: 105031| Number of nodes= 13
Component id: 192473| Number of nodes= 11
Component id: 118607| Number of nodes= 37
Component id: 20779| Number of nodes= 12
Component id: 223980| Number of nodes= 8
Component id: 21695| Number of nodes= 20
Component id: 140813| Number of nodes= 20
Component id: 144930| Number of nodes= 19
Component id: 187829| Number of nodes= 3
Component id: 161520| Number of nodes= 14
Component id: 218961| Number of nodes= 3
Component id: 119| Number of nodes= 9
Component id: 206333| Number of nodes= 6
Component id: 173970| Number of nodes= 11
Component id: 164| Number of nodes= 5
Component id: 156662| Number of nodes= 8
Component id: 11314| Number of nodes= 28
Component id: 243135| Number of nodes= 8
Component id: 219931| Number of nodes= 10
Component id: 186992| Number of nodes= 6
Component id: 399424| Number of nodes= 3
Component id: 102315| Number of nodes= 16
Component id: 102| Number of nodes= 5
Component id: 149930| Number of nodes= 20
Component id: 461| Number of nodes= 3
Component id: 127959| Number of nodes= 3
Component id: 513| Number of nodes= 5
Component id: 338| Number of nodes= 4
Component id: 279176| Number of nodes= 4
Component id: 502| Number of nodes= 3
Component id: 138| Number of nodes= 6
Component id: 321918| Number of nodes= 5
Component id: 145654| Number of nodes= 6
Component id: 167| Number of nodes= 2
Component id: 570| Number of nodes= 3
Component id: 411| Number of nodes= 4
Component id: 272| Number of nodes= 2
Component id: 12| Number of nodes= 2
Component id: 453| Number of nodes= 2
Component id: 270| Number of nodes= 3
Component id: 158318| Number of nodes= 3
Component id: 207025| Number of nodes= 2
Component id: 261310| Number of nodes= 2
Component id: 160| Number of nodes= 2
Component id: 189| Number of nodes= 2
Component id: 365| Number of nodes= 2
Component id: 582910| Number of nodes= 4
Component id: 509| Number of nodes= 2
Component id: 361| Number of nodes= 2
Component id: 100049| Number of nodes= 244
Component id: 102817| Number of nodes= 248
Component id: 101793| Number of nodes= 170
Comp

In [11]:
r=sc.parallelize([("A","B"),("B","C"),("B","D"),("D","E"),("F","G"),("G","H"),("A","H"),("G","H"),("I","H")],2)
mapJob = r.flatMap(lambda e: (e,e[::-1]))
    #print(mapJob.collect())
    
    #Secondary Sort
n=3
rddSS= (mapJob
  .keyBy(lambda kv: (kv[0], kv[1][0]))  # Create temporary composite key
  .repartitionAndSortWithinPartitions(numPartitions=n, partitionFunc=partitioner(n), ascending=True))
  #.map(lambda x: x[1]))  # Drop key (note: there is no partitioner set anymore)
rddSS.glom().collect()


In [12]:
def unpair2(entry):
    return entry[0][0], entry[0][1]
  
def sorted_group(lines):
    return itertools.groupby(lines, key=lambda x: x[0])

In [13]:
unpairedRDD = rddSS.map(unpair2, preservesPartitioning=True)

unpairedRDD.glom().collect()

In [14]:
groupedRDD = unpairedRDD.mapPartitions(sorted_group, preservesPartitioning=True)
groupedRDD.glom().collect()

In [15]:
    # CCF-iterate (REDUCE)
accum = sc.accumulator(0)
reduceJob = groupedRDD.flatMap(CCF_Iterate_reduce_SS_shuffle_test)
reduceJob.collect()